# Example Topic Analysis using LDA in SciKit Learn

A simple example of how to perform Latent Drichlet Allocation to extract topics from a corpus of text. The example uses the [SciKit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) implementation of LDA on a sample of the [20 newsgroups dataset (classification)](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html). Vectorisation is performed using [Count Vectoriser](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html).

In [1]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups


import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

## Load the dataset

In [2]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
documents[1]
print(len(documents))

11314


## Convert to Term-Document Matrix

In [3]:
# restrict to returning the top max_features ordered by term frequency across the corpus
no_features = 1000

# Count Vectoriser
# There is some pre-processing we can achieve through this, e.g.,

# max_df - When building the vocabulary ignore terms that have a document frequency 
#          strictly higher than the given threshold - percentage
# min_df - When building the vocabulary ignore terms that have a document frequency 
#          strictly lower than the given threshold. This value is also called cut-off in the literature
# token_pattern - Regular expression denoting what constitutes a “token”, e.g., token_pattern = r'\b[a-zA-Z]{3,}\b'
#                 would only include words with only letters with a min length of 3


tf_vectorizer = CountVectorizer(max_df=0.95,
                                min_df=10, 
                                max_features=no_features, 
                                stop_words='english',
                                strip_accents = 'unicode',
                                lowercase = True
                               )

tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

Lets look at the matrix - it is sparse (many 0 entries) and is of dimensions DxN where D is the number of documents and N is the number of feautures or tokens (words).

In [4]:
tf

<11314x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 286645 stored elements in Compressed Sparse Row format>

In [5]:
print(tf.shape)

(11314, 1000)


Lets look at the matrix itself - it is sparse with many 0s

In [6]:
tf.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

Take a look at some of the tokens:

In [7]:
tf_feature_names[200:210]

['cause',
 'cd',
 'center',
 'certain',
 'certainly',
 'chance',
 'change',
 'changed',
 'changes',
 'check']

## Perform LDA
We have to define the number of topics. Here we know that we are expecting 20 different news groups, so we look for 20. In other dtasets we may need to run several different numbers of topics and see what returns the most useful. 

In [8]:
no_topics = 20

# Run LDA
lda = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

## Show Topics
Some topics make sense - some dont! 

#### Question - Can you think of ways to improve this?
#### What preprocessing would you include? Have a play with the CountVectoriser settings above and re-run.

In [9]:
def display_topics(model, feature_names, no_top_words):
   for topic_idx, topic in enumerate(model.components_):
       print("Topic %d:" % (topic_idx))
       print(" ".join([feature_names[i]
                       for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
display_topics(lda, tf_feature_names, no_top_words)


Topic 0:
00 50 25 price sale new 20 shipping 10 15
Topic 1:
time high good problems low like don run better soon
Topic 2:
use point case question make does people used possible way
Topic 3:
10 11 14 12 17 16 15 13 18 20
Topic 4:
card bit memory color video speed drivers mode data 16
Topic 5:
thanks window db does application know help hi widget advance
Topic 6:
key chip encryption keys clipper security privacy public use algorithm
Topic 7:
edu file com available ftp files program version image mail
Topic 8:
game team games year play season hockey league players win
Topic 9:
god jesus people believe does say christian bible life think
Topic 10:
drive windows disk scsi use mac problem dos hard pc
Topic 11:
car cars church com engine article used perfect jim true
Topic 12:
book good books ago water years left best year radio
Topic 13:
ax max b8f g9v a86 pl 145 1d9 0t 34u
Topic 14:
don just like know think ve really ll going people
Topic 15:
output university current use section medical gr

## Assign topic probabilities to the documents

In [10]:
doc_topic_distrib = lda.transform(tf)

This assigns the probability that the document belongs to each class - so lets look at the first document.

In [11]:
doc_topic_distrib[1]

array([0.00151515, 0.00151515, 0.00151515, 0.00151515, 0.00151515,
       0.00151515, 0.00151515, 0.12000064, 0.00151515, 0.23673105,
       0.00151515, 0.09264141, 0.00151515, 0.00151515, 0.52638447,
       0.00151515, 0.00151515, 0.00151515, 0.00151515, 0.00151515])

We can pick out the top matching topics for that document.

## Visualising the Model

In [13]:
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

/anaconda3/envs/dp_spacy/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13    -0.349890  0.177826       1        1  14.841506
14     0.205417  0.125730       2        1  12.979644
9      0.206229  0.103306       3        1   8.153746
7     -0.018347 -0.245284       4        1   8.137915
2      0.198758  0.002409       5        1   7.882985
18     0.143029  0.155968       6        1   5.680273
17     0.115182  0.106720       7        1   4.779913
10     0.094218 -0.119279       8        1   4.534150
1      0.169371  0.046925       9        1   4.439766
3     -0.272913  0.028525      10        1   4.302089
16    -0.043539 -0.153703      11        1   4.296462
4      0.003986 -0.188010      12        1   3.347076
6      0.058954 -0.127998      13        1   3.071173
5      0.015503 -0.196737      14        1   2.985858
8     -0.014146  0.177300      15        1   2.868797
12     0.080392  0.103264      16        1   2.360889
15    -0.057436 -0.169865      17        1   2.178489
0     -0.217757 -0.052288      18        1   1.909796
11    -0.047625  0.112571      19        1   0.962971
19    -0.269386  0.112618      20        1   0.286503, topic_info=    Category          Freq        Term         Total  loglift  logprob
146  Default  61244.000000          ax  61244.000000  30.0000  30.0000
584  Default   4569.000000         max   4569.000000  29.0000  29.0000
416  Default   1881.000000         god   1881.000000  28.0000  28.0000
0    Default   1171.000000          00   1171.000000  27.0000  27.0000
509  Default   1268.000000         key   1268.000000  26.0000  26.0000
233  Default   1447.000000         com   1447.000000  25.0000  25.0000
378  Default   1709.000000        file   1709.000000  24.0000  24.0000
334  Default   2369.000000         edu   2369.000000  23.0000  23.0000
192  Default    839.000000         car    839.000000  22.0000  22.0000
847  Default   1281.000000       space   1281.000000  21.0000  21.0000
671  Default   4082.000000      people   4082.000000  20.0000  20.0000
894  Default   1251.000000      thanks   1251.000000  19.0000  19.0000
932  Default   3225.000000         use   3225.000000  18.0000  18.0000
419  Default   2478.000000        good   2478.000000  17.0000  17.0000
323  Default   1047.000000       drive   1047.000000  16.0000  16.0000
8    Default   1636.000000          10   1636.000000  15.0000  15.0000
516  Default   3457.000000        know   3457.000000  14.0000  14.0000
34   Default   1022.000000          25   1022.000000  13.0000  13.0000
319  Default   3877.000000         don   3877.000000  12.0000  12.0000
400  Default    833.000000        game    833.000000  11.0000  11.0000
995  Default   1497.000000        year   1497.000000  10.0000  10.0000
316  Default   2754.000000        does   2754.000000   9.0000   9.0000
884  Default    779.000000        team    779.000000   8.0000   8.0000
786  Default   1717.000000        said   1717.000000   7.0000   7.0000
973  Default   1223.000000     windows   1223.000000   6.0000   6.0000
972  Default    805.000000      window    805.000000   5.0000   5.0000
422  Default   1238.000000  government   1238.000000   4.0000   4.0000
508  Default   3698.000000        just   3698.000000   3.0000   3.0000
163  Default   1228.000000         bit   1228.000000   2.0000   2.0000
147  Default   1353.000000         b8f   1353.000000   1.0000   1.0000
..       ...           ...         ...           ...      ...      ...
813  Topic20     89.002797      serial    331.989045   4.5387  -2.8736
584  Topic20    486.210841         max   4569.377676   3.6147  -1.1756
291  Topic20     23.924974      define    242.599109   3.5387  -4.1873
662  Topic20      8.224891       paper    240.754938   2.4786  -5.2551
42   Topic20      7.293608         300    228.863523   2.4090  -5.3752
61   Topic20      5.549052         500    247.855453   2.0560  -5.6486
669  Topic20      6.581894          pc    657.437495   1.2511  -5.4779
